In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install bioinfokit
!pip install seaborn
!pip install seaborn[stats]

from IPython.display import display, HTML
from pandas.core.algorithms import duplicated
from pandas.errors import AccessorRegistrationWarning
from scipy import stats as st
from scipy.stats import ttest_ind_from_stats
from scipy.stats import combine_pvalues
from bioinfokit import analys, visuz
import math
import os
import glob
import pandas as pd
import re
import numpy as np
from numpy.ma.core import mean
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
from sklearn.impute import KNNImputer

In [ ]:
# Parameter Settings
significance_cutoff = np.log10(0.025)*-10
FC = np.log2(1.5)
sex = 'Male-Female'
fraction = 'BCM'

In [ ]:
# NORMALIZATION AND STATS
from numpy.core.shape_base import atleast_3d
# Parameters
where_are_the_files = f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/Peaks_DB_Data'
os.chdir(where_are_the_files)
all_file_names = [i for i in glob.glob('*.{}'.format('csv'))]

E2vE3_PVFC_DF = pd.DataFrame()
E4vE3_PVFC_DF = pd.DataFrame()
E4vE2_PVFC_DF = pd.DataFrame()

for dataSet in all_file_names:
  fileName = dataSet.strip('.csv')
  dataDF = pd.read_csv(dataSet)

  # Read CSV, set the 'Accession' as the index, filter out proteins with <2 unique peptides, and filter out non-area data.
  dataDF = dataDF.set_index('Accession')
  dataDF = dataDF.loc[(dataDF['#Unique'] >= 2),]
  dataDF = dataDF.loc[dataDF['Top'] == True,]
  dataDF = dataDF.filter(regex='Area')

  # Replace 0 with NaN, create allele specific dataframes, count non-zero values, remove proteins with >1 missing value
  dataDF = dataDF.replace(np.NaN,0)
  dataDF['nonZero_Count'] = (dataDF != 0).astype(int).sum(axis=1)

  E2_DF = dataDF.filter(regex='A2').copy()
  E3_DF = dataDF.filter(regex='A3').copy()
  E4_DF = dataDF.filter(regex='A4').copy()

  cols_E2_DF = len(E2_DF.axes[1])
  cols_E3_DF = len(E3_DF.axes[1])
  cols_E4_DF = len(E4_DF.axes[1])

  E2_DF['E2_nonZero_Count'] = (E2_DF != 0).astype(int).sum(axis=1)
  E3_DF['E3_nonZero_Count'] = (E3_DF != 0).astype(int).sum(axis=1)
  E4_DF['E4_nonZero_Count'] = (E4_DF != 0).astype(int).sum(axis=1)

  E2_DF = E2_DF.loc[(E2_DF['E2_nonZero_Count'] >= (cols_E2_DF-1)),]
  E3_DF = E3_DF.loc[(E3_DF['E3_nonZero_Count'] >= (cols_E3_DF-1)),]
  E4_DF = E4_DF.loc[(E4_DF['E4_nonZero_Count'] >= (cols_E4_DF-1)),]

  E2vE3_comparison_DF = pd.concat([E2_DF,E3_DF],axis=1).dropna().filter(regex='Area').copy()
  E4vE3_comparison_DF = pd.concat([E4_DF,E3_DF],axis=1).dropna().filter(regex='Area').copy()
  E4vE2_comparison_DF = pd.concat([E4_DF,E2_DF],axis=1).dropna().filter(regex='Area').copy()

  E4vE3vE2_comparison_DF = pd.concat([E4_DF,E3_DF,E2_DF],axis=1).dropna().filter(regex='Area').copy()

  def slope_normalize(input_df):
    # Creat allele comparison dataframes, log2 normalize and replace -inf values with NaN.

    df_to_normalize = input_df.copy()
    df_to_normalize = np.log2(df_to_normalize)
    df_to_normalize = df_to_normalize.replace(-np.inf, np.NaN)

    # Normalize Data by the Average
    for sample in df_to_normalize.columns:
      df_to_normalize[sample] = df_to_normalize[sample] - np.mean(df_to_normalize[sample])
    df_to_normalize['protein_avg'] = np.mean(df_to_normalize,axis=1)

    # Normalize Data by the Slope
    for sample in df_to_normalize.columns:
      x= df_to_normalize['protein_avg']
      y= df_to_normalize[sample]
      mask = ~np.isnan(x) & ~np.isnan(y)
      slope = st.linregress(x[mask], y[mask])[0]
      df_to_normalize[sample] = df_to_normalize[sample]/slope
    normalized_DF = df_to_normalize.filter(regex='Area')

    # Impute missing values using KNN imputer
    imputer = KNNImputer(n_neighbors=2)
    normalized_DF = pd.DataFrame(imputer.fit_transform(normalized_DF),columns=normalized_DF.columns,index=normalized_DF.index)
    x = normalized_DF.plot.density(figsize=(10, 10))

    return(normalized_DF)

  E4vE3vE2_norm_DF = slope_normalize(E4vE3vE2_comparison_DF)

  E2vE3_norm_DF = E4vE3vE2_norm_DF.filter(regex='A2|A3').copy()
  E4vE3_norm_DF = E4vE3vE2_norm_DF.filter(regex='A4|A3').copy()
  E4vE2_norm_DF = E4vE3vE2_norm_DF.filter(regex='A4|A2').copy()

  E2vE3_norm_DF.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/Normalized_Peaks_Data/{fileName}_E2vE3_Normalized_DataSet.csv',index=True)
  E4vE3_norm_DF.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/Normalized_Peaks_Data/{fileName}_E4vE3_Normalized_DataSet.csv',index=True)
  E4vE2_norm_DF.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/Normalized_Peaks_Data/{fileName}_E4vE2_Normalized_DataSet.csv',index=True)

  # Perform T-test and calculate PV
  def stats_calc(input_df,comparison=None):
    if comparison == 'E2vE3':
      allele_a = 'A3'
      allele_b = 'A2'
    elif comparison == 'E4vE3':
      allele_a = 'A3'
      allele_b = 'A4'
    elif comparison == 'E4vE2':
      allele_a = 'A2'
      allele_b = 'A4'

    comparison_DF_b = input_df.filter(regex=allele_b).copy()
    comparison_DF_a = input_df.filter(regex=allele_a).copy()

    comparison_p_val_dict = {}
    comparison_FC_dict = {}

    for protein in input_df.index.to_list():
      #ApoEb vs ApoEa P-value and FC
      a = comparison_DF_b.loc[protein,]
      b = comparison_DF_a.loc[protein,]

      f_PV = st.f_oneway(a, b)[1]

      if f_PV > 0.05:
        comparison_p_val = st.ttest_ind(a,b, equal_var=True, alternative='two-sided',nan_policy='omit')[1]
        comparison_p_val_dict[protein] = comparison_p_val

      elif f_PV < 0.05:
        comparison_p_val = st.ttest_ind(a,b, equal_var=False, alternative='two-sided',nan_policy='omit')[1]
        comparison_p_val_dict[protein] = comparison_p_val

# Calculate FC
      comparison_FC = (np.mean(b)) - (np.mean(a))
      comparison_FC = (np.mean(b)) - (np.mean(a))

      comparison_FC_dict[protein] = comparison_FC

    comparison_df_PV = pd.DataFrame.from_dict(comparison_p_val_dict.items()).rename(columns={0 : 'Accession',1 : f'{comparison}_PV'}).set_index('Accession')
    comparison_df_FC = pd.DataFrame.from_dict(comparison_FC_dict.items()).rename(columns={0 : 'Accession',1 : f'{comparison}_FC'}).set_index('Accession')

    comparison_df_PV = pd.melt(comparison_df_PV,ignore_index=False).rename(columns={'value' : 'P_val'})
    comparison_df_PV = comparison_df_PV.drop(columns='variable')

    comparison_df_FC = pd.melt(comparison_df_FC,ignore_index=False).rename(columns={'value' : 'Fold_change'})

    comparison_PVFC = pd.concat([comparison_df_PV,comparison_df_FC],axis=1)

    comparison_PVFC['-log10(PV)'] = np.log10(comparison_PVFC['P_val'])*-10
    return(comparison_PVFC)

  E2vE3_DF = stats_calc(E2vE3_norm_DF,comparison='E2vE3')
  E4vE3_DF = stats_calc(E4vE3_norm_DF,comparison='E4vE3')
  E4vE2_DF = stats_calc(E4vE2_norm_DF,comparison='E4vE2')

  E2vE3_PVFC_DF = pd.concat([E2vE3_PVFC_DF,E2vE3_DF])
  E4vE3_PVFC_DF = pd.concat([E4vE3_PVFC_DF,E4vE3_DF])
  E4vE2_PVFC_DF = pd.concat([E4vE2_PVFC_DF,E4vE2_DF])

  # Calculate BH P-value correction for the DataSet
  def performBH_correction(input_df):
    input_df['Benjamini_Hochberg_pval'] = None
    input_df = input_df.reset_index()
    #sort cleaned_df by pvalue jc mod
    input_df = input_df.sort_values(by='P_val')
    input_df = input_df.reset_index(drop=True) #sort keeps the origional index value so you need to re-index to use it in the BH calc
    #calculate benjamini_hochberg correction as (rank/total numer of tests)*probability of false positive jc mod
    total_rows = len(input_df.index)
    for row in input_df.itertuples():
      BH_pval = ((row.Index+1)/total_rows)*0.25
      input_df.at[row.Index, 'Benjamini_Hochberg_pval'] = BH_pval
    input_df['-10*log10(BH)'] = np.log10(input_df['Benjamini_Hochberg_pval'].astype(float))*-10
    return(input_df)

  E2vE3_DF = performBH_correction(E2vE3_DF)
  E4vE3_DF = performBH_correction(E4vE3_DF)
  E4vE2_DF = performBH_correction(E4vE2_DF)

  E2vE3_DF.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/PVFC/E2vE3_PVFC_{fileName}.csv',index=True)
  E4vE3_DF.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/PVFC/E4vE3_PVFC_{fileName}.csv',index=True)
  E4vE2_DF.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/PVFC/E4vE2_PVFC_{fileName}.csv',index=True)

  # Create Volcano plots for the comparisons
  def plotVP(input_df,comparison=None):
    df = input_df
    sns.set_style('white')
    plt.figure(figsize=(10, 10))
    sns.set(style='white', context='talk')
    plt.axvline(x=FC, ymin=0, ymax=1, linestyle=':',color='gray')
    plt.axvline(x=-FC, ymin=0, ymax=1, linestyle=':',color='gray')
    plt.axhline(y=significance_cutoff, xmin=0, xmax=1, linestyle=':',color='gray')
    plt.xlabel("Fold_change")
    plt.ylabel("-10*log10(BH)")
    plt.title(f'P-value vs. Fold Change',fontsize=30)
    plt.xlim(-6, 6)

    df['cutoff'] = ''

    df.loc[((df['Fold_change'] < FC) | (df['Fold_change'] > -FC)) & (df['-10*log10(BH)'] < significance_cutoff),['cutoff']] = 100
    df.loc[((df['Fold_change'] < FC) | (df['Fold_change'] > -FC)) & (df['-10*log10(BH)'] >= significance_cutoff),['cutoff']] = 100
    df.loc[((df['Fold_change'] >= FC)) & (df['-10*log10(BH)'] >= significance_cutoff),['cutoff']] = 250
    df.loc[((df['Fold_change'] <= -FC)) & (df['-10*log10(BH)'] >= significance_cutoff),['cutoff']] = 15

    VP = sns.scatterplot(x='Fold_change',
                        y='-10*log10(BH)',
                        data=df,
                        legend=False,
                        palette='cividis',
                        hue='cutoff',
                        s = 350,
                        alpha=0.8)

    plt.savefig(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/Figures/{fileName}_{comparison}_VP.svg',format="svg",transparent=True)

    return(VP)

  plotVP(E2vE3_DF,'E2vE3')
  plotVP(E4vE3_DF,'E4vE3')
  plotVP(E4vE2_DF,'E4vE2')

  continue

In [ ]:
where_are_the_files = f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/PVFC'
os.chdir(where_are_the_files)
all_file_names = [i for i in glob.glob('*.{}'.format('csv'))]

primary_E2vE3_PVFC = pd.DataFrame()
primary_E4vE3_PVFC = pd.DataFrame()
primary_E4vE2_PVFC = pd.DataFrame()

for dataSet in all_file_names:
  fileName = dataSet.strip('.csv')
  PVFC_DF = pd.read_csv(dataSet).set_index('Accession').filter(regex='P_val|Fold_change')
  # stdev_ = PVFC_DF['Fold_change'].std()
  # PVFC_DF['Fold_change'] = PVFC_DF['Fold_change'] / 2**stdev_
  PVFC_DF['Fold_change'] = PVFC_DF['Fold_change']

  # display(PVFC_DF)
  if 'E2vE3' in dataSet:
    primary_E2vE3_PVFC = pd.concat([primary_E2vE3_PVFC,PVFC_DF])
  elif 'E4vE3' in dataSet:
    primary_E4vE3_PVFC = pd.concat([primary_E4vE3_PVFC,PVFC_DF])
  elif 'E4vE2' in dataSet:
    primary_E4vE2_PVFC = pd.concat([primary_E4vE2_PVFC,PVFC_DF])
  continue

In [ ]:
primary_E2vE3 = primary_E2vE3_PVFC.copy()
primary_E4vE3 = primary_E4vE3_PVFC.copy()
primary_E4vE2 = primary_E4vE2_PVFC.copy()

In [ ]:
import numpy as np
from scipy.stats import combine_pvalues

def combine_PV_and_FC(input_PVFC_DF):
  protein_list = input_PVFC_DF.index.drop_duplicates()

  pv_df = input_PVFC_DF.copy().filter(regex='P_val')
  PV_dict = {}
  combined_PV_dict = {}

  for protein in protein_list:
    vals = pv_df['P_val'][protein].tolist()
    pv_array = np.array(vals, ndmin=1)
    combined_pv = combine_pvalues(pv_array,method='mudholkar_george',weights=None)[1]
    PV_dict[protein] = vals
    combined_PV_dict[protein] = combined_pv

# Calculate Average fold-change
  fc_df = input_PVFC_DF.copy().filter(regex='Fold_change')

  fc_df = 2**(fc_df)
  fc_df = fc_df.reset_index()
  fc_df = fc_df.groupby(by='Accession',as_index=True).mean()
  # fc_df =(fc_df['Fold_change'])
  fc_df = np.log2(fc_df['Fold_change'])

  clean_PV_DF = pd.DataFrame.from_dict(combined_PV_dict.items()).rename(columns={0 : 'Accession',1 : 'fisher_combined_PV'}).set_index('Accession')
  cleaned_PVFC_DF = pd.concat([clean_PV_DF,fc_df],axis=1)
  cleaned_PVFC_DF['-10log10(comb_PV)'] = np.log10(cleaned_PVFC_DF['fisher_combined_PV'])*-10

  def performBH_correction(input_df):
    input_df['Benjamini_Hochberg_pval'] = None
    input_df = input_df.reset_index()
    #sort cleaned_df by pvalue jc mod
    input_df = input_df.sort_values(by='fisher_combined_PV')
    input_df = input_df.reset_index(drop=True) #sort keeps the origional index value so you need to re-index to use it in the BH calc
    #calculate benjamini_hochberg correction as (rank/total numer of tests)*probability of false positive jc mod
    total_rows = len(input_df.index)
    for row in input_df.itertuples():
      BH_pval = ((row.Index+1)/total_rows)*0.25
      input_df.at[row.Index, 'Benjamini_Hochberg_pval'] = BH_pval
    input_df['-10*log10(BH)'] = np.log10(input_df['Benjamini_Hochberg_pval'].astype(float))*-10
    return(input_df)

  cleaned_PVFC_DF = performBH_correction(cleaned_PVFC_DF).set_index('Accession')

  def plotVP(input_df):
    df = input_df
    sns.set_style('white')
    plt.figure(figsize=(10, 10))
    sns.set(style='white', context='talk')
    plt.axvline(x=FC, ymin=0, ymax=1, linestyle=':',color='gray')
    plt.axvline(x=-FC, ymin=0, ymax=1, linestyle=':',color='gray')
    plt.axhline(y=significance_cutoff, xmin=0, xmax=1, linestyle=':',color='gray')
    plt.xlabel("Fold_change")
    plt.ylabel("-10*log10(BH)")
    plt.title(f'P-value vs. Fold Change',fontsize=30)
    # plt.xlim(-4,4)

    df['cutoff'] = ''

    df.loc[((df['Fold_change'] < FC) | (df['Fold_change'] > -FC)) & (df['-10*log10(BH)'] < significance_cutoff),['cutoff']] = '-'
    df.loc[((df['Fold_change'] < FC) | (df['Fold_change'] > -FC)) & (df['-10*log10(BH)'] > significance_cutoff),['cutoff']] = '-'
    df.loc[((df['Fold_change'] >= FC)) & (df['-10*log10(BH)'] >= significance_cutoff),['cutoff']] = 'sigUp'
    df.loc[((df['Fold_change'] <= -FC)) & (df['-10*log10(BH)'] >= significance_cutoff),['cutoff']] = 'sigDown'

    sigUp = len(df.loc[(df['cutoff'] == 'sigUp'),['cutoff']].index)
    sigDown = len(df.loc[(df['cutoff'] == 'sigDown'),['cutoff']].index)

    print(sigUp,sigDown)

    VP = sns.scatterplot(x='Fold_change',
                        y='-10*log10(BH)',
                        data=df,
                        legend=False,
                        palette='cividis',
                        hue='cutoff',
                        s = 350,
                        alpha=0.8)
    return(VP)

  plotVP(cleaned_PVFC_DF)

  # primary_E2vE3_PVFC.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/PVFC/E2vE3_primary_PVFC.csv',index=True)
  # primary_E4vE3_PVFC.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/PVFC/E4vE3_primary_PVFC.csv',index=True)

  display(cleaned_PVFC_DF)

  return(cleaned_PVFC_DF)

primary_E2vE3_stats = combine_PV_and_FC(primary_E2vE3_PVFC)
primary_E4vE3_stats = combine_PV_and_FC(primary_E4vE3_PVFC)
primary_E4vE2_stats = combine_PV_and_FC(primary_E4vE2_PVFC)

primary_E2vE3_stats.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/primary_PVFC/E2vE3_primary_PVFC.csv',index=True)
primary_E4vE3_stats.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/primary_PVFC/E4vE3_primary_PVFC.csv',index=True)
primary_E4vE2_stats.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/primary_PVFC/E4vE2_primary_PVFC.csv',index=True)


In [ ]:
E2vE3_primary_PVFC_df = pd.read_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/primary_PVFC/E2vE3_primary_PVFC.csv').set_index('Accession').filter(regex='Fold').rename(columns={'Fold_change':'E2vE3_Fold_change'})
E4vE3_primary_PVFC_df = pd.read_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/primary_PVFC/E4vE3_primary_PVFC.csv').set_index('Accession').filter(regex='Fold').rename(columns={'Fold_change':'E4vE3_Fold_change'})
E4vE2_primary_PVFC_df = pd.read_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/primary_PVFC/E4vE2_primary_PVFC.csv').set_index('Accession').filter(regex='Fold').rename(columns={'Fold_change':'E4vE2_Fold_change'})

All_genotype_Fold_changes = pd.concat([E2vE3_primary_PVFC_df,E4vE3_primary_PVFC_df,E4vE2_primary_PVFC_df],axis=1)

All_genotype_Fold_changes.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/All_Genotype_Fold_Changes/All_genotype_Fold_changes.csv',index=True)

def plotVP(input_df):
  df = input_df
  sns.set_style('white')
  plt.figure(figsize=(10, 10))
  sns.set(style='white', context='talk')
  plt.axvline(x=FC, ymin=0, ymax=1, linestyle=':',color='gray')
  plt.axvline(x=-FC, ymin=0, ymax=1, linestyle=':',color='gray')
  plt.axhline(y=FC, xmin=0, xmax=1, linestyle=':',color='gray')
  plt.axhline(y=-FC, xmin=0, xmax=1, linestyle=':',color='gray')
  # plt.xlabel("Fold_change")
  # plt.ylabel("-10*log10(BH)")
  plt.title(f'Fold Change vs. Fold Change',fontsize=30)
  # plt.xlim(-3, 3)

  VP = sns.scatterplot(x='E2vE3_Fold_change',
                      y='E4vE3_Fold_change',
                      data=All_genotype_Fold_changes,
                      legend=False,
                      palette='cividis',
                      s = 350,
                      alpha=0.8)
  return(VP)

plotVP(All_genotype_Fold_changes)


In [ ]:
primary_E2vE3_stats.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/primary_PVFC/E2vE3_primary_PVFC.csv',index=True)
primary_E4vE3_stats.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/primary_PVFC/E4vE3_primary_PVFC.csv',index=True)
primary_E4vE2_stats.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/primary_PVFC/E4vE2_primary_PVFC.csv',index=True)

In [ ]:
primary_E2vE3_stats = pd.read_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/primary_PVFC/E2vE3_primary_PVFC.csv').set_index('Accession')
primary_E4vE3_stats = pd.read_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/primary_PVFC/E4vE3_primary_PVFC.csv').set_index('Accession')
primary_E4vE2_stats = pd.read_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/primary_PVFC/E4vE2_primary_PVFC.csv').set_index('Accession')

In [ ]:
colors = ['#306998','#646464','#FFD43B']
custom_pal = sns.color_palette(colors)

change_cutoff = np.log2(1.5)

def plotVP(input_df,
           plot_title=None,
           FC_column_name=None,
           PV_column_name=None,
           fractions=None,
           comparison=None,
           style=None):

  df = input_df.copy()

  plt.figure(figsize=(10, 10))

  custom_params = {'axes.linewidth': 3}

  sns.set(style='ticks', context='paper',font='sans-serif', font_scale=1, color_codes=True,rc=custom_params)

  plt.axvline(x=change_cutoff, ymin=0, ymax=1, linestyle=':',color='gray')
  plt.axvline(x=-change_cutoff, ymin=0, ymax=1, linestyle=':',color='gray')
  plt.axhline(y=significance_cutoff, xmin=0, xmax=1, linestyle=':',color='gray')
  plt.xlabel("log2(fold change)", fontweight ='bold', size=20)
  plt.ylabel("-10Log(BH-corrected p-value)", fontweight ='bold',size=20)
  # plt.title(f'{plot_title} P-value vs. Fold Change',fontsize=30)
  plt.tick_params(axis='both', which='major', labelsize=15,length = 5,color = 'black',width =3)

  plt.xlim(-4,4)

  df['cutoff'] = ''

  sigUp_count = len(df.loc[((df[FC_column_name] >= change_cutoff)) & (df[PV_column_name] >= significance_cutoff),['cutoff']].index)
  sigDown_count = len(df.loc[((df[FC_column_name] <= -change_cutoff)) & (df[PV_column_name] >= significance_cutoff),['cutoff']].index)
  no_change1 = len(df.loc[((df[FC_column_name] < change_cutoff) | (df[FC_column_name] > -change_cutoff)) & (df[PV_column_name] < significance_cutoff),['cutoff']].index)
  no_change2 = len(df.loc[((df[FC_column_name] < change_cutoff) | (df[FC_column_name] > -change_cutoff)) & (df[PV_column_name] >= significance_cutoff),['cutoff']].index)

  no_change_total = (no_change1 + no_change2) - (sigUp_count + sigDown_count)

  print(sigUp_count,sigDown_count,no_change1)

  df.loc[((df[FC_column_name] < change_cutoff) | (df[FC_column_name] > -change_cutoff)) & (df[PV_column_name] < significance_cutoff),['cutoff']] = f"━{str(no_change_total)}"
  df.loc[((df[FC_column_name] < change_cutoff) | (df[FC_column_name] > -change_cutoff)) & (df[PV_column_name] >= significance_cutoff),['cutoff']] = f"━{str(no_change_total)}"
  df.loc[((df[FC_column_name] >= change_cutoff)) & (df[PV_column_name] >= significance_cutoff),['cutoff']] = f"⬆{str(sigUp_count)}"
  df.loc[((df[FC_column_name] <= -change_cutoff)) & (df[PV_column_name] >= significance_cutoff),['cutoff']] = f"⬇{str(sigDown_count)}"

  display(df)

  markers = {"BM": "p", "BC": "o"}

  VP = sns.scatterplot(x=FC_column_name,
                      y=PV_column_name,
                      data=df,
                      legend=True,
                      palette=custom_pal,
                      hue='cutoff',
                      s = 200,
                      alpha=0.9,
                      edgecolor="black",
                      linewidth=0.8,
                      hue_order = [f"⬆{str(sigUp_count)}",f"━{str(no_change_total)}",f"⬇{str(sigDown_count)}"],
                      markers = markers,
                      style=style)

  plt.legend(loc=1, prop={'size':15},title=None,title_fontsize=20,markerscale=3)
  sns.despine(top=True, right=True, left=False, bottom=False, offset=None, trim=False)
  plt.savefig(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{sex}/Figures/{sex}_{comparison}_{fractions}_primary_VP.svg',format="svg",transparent=True)

  return(VP)

In [ ]:
def auto_scaling(df):
  df_mean = np.mean(df[f'Fold_change'])
  df_std = np.std(df[f'Fold_change'])
  df[f'Fold_change'] = df[f'Fold_change'] - df_mean
  df[f'Fold_change'] = df[f'Fold_change'] / df_std
  return(df)

def range_scaling(df):
  df_min = np.min(df[f'Fold_change'])
  df_max = np.max(df[f'Fold_change'])
  df_mean = np.mean(df[f'Fold_change'])
  df[f'Fold_change'] = df[f'Fold_change'] - df_mean
  df[f'Fold_change'] = df[f'Fold_change'] / (df_max - df_min)
  df[f'Fold_change'] = df[f'Fold_change']*10
  return(df)

def pareto_scaling(df):
  df_mean = np.mean(df[f'Fold_change'])
  df_std = np.std(df[f'Fold_change'])
  df[f'Fold_change'] = df[f'Fold_change'] - df_mean
  df[f'Fold_change'] = df[f'Fold_change'] /  np.sqrt(df_std)
  return(df)

def vast_scaling(df):
  Rate_FC_DF_mean = np.mean(Rate_FC_DF[f'{comparison}_rate_FC'])
  Rate_FC_DF_std = np.std(Rate_FC_DF[f'{comparison}_rate_FC'])
  Rate_FC_DF[f'{comparison}_rate_FC'] = Rate_FC_DF[f'{comparison}_rate_FC'] - Rate_FC_DF_mean
  Rate_FC_DF[f'{comparison}_rate_FC'] = Rate_FC_DF[f'{comparison}_rate_FC'] / (Rate_FC_DF_std)
  Rate_FC_DF[f'{comparison}_rate_FC'] = Rate_FC_DF[f'{comparison}_rate_FC'] * (Rate_FC_DF_mean/Rate_FC_DF_std)

def level_scaling(df):
  Rate_FC_DF_mean = np.mean(Rate_FC_DF[f'{comparison}_rate_FC'])
  Rate_FC_DF[f'{comparison}_rate_FC'] = Rate_FC_DF[f'{comparison}_rate_FC'] - Rate_FC_DF_mean
  Rate_FC_DF[f'{comparison}_rate_FC'] = Rate_FC_DF[f'{comparison}_rate_FC'] / (Rate_FC_DF_mean)

In [ ]:
print('primary_E2vE3_stats')
print(len(primary_E2vE3_stats.index))
plotVP(primary_E2vE3_stats,
           plot_title='Test',
           FC_column_name='Fold_change',
           PV_column_name='-10*log10(BH)',
           fractions='BCM',
           comparison='E2vE3',
           style=None)

print('primary_E4vE3_stats')
print(len(primary_E4vE3_stats.index))
plotVP(primary_E4vE3_stats,
           plot_title='Test',
           FC_column_name='Fold_change',
           PV_column_name='-10*log10(BH)',
           fractions='BCM',
           comparison='E4vE3',
           style=None)

print('primary_E4vE2_stats')
print(len(primary_E4vE2_stats.index))
plotVP(primary_E4vE2_stats,
           plot_title='Test',
           FC_column_name='Fold_change',
           PV_column_name='-10*log10(BH)',
           fractions='BCM',
           comparison='E4vE2',
           style=None)

In [ ]:
# # Prepare the dataframe with the fold changes for each protein in each comparison
comparison_DF = pd.read_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{FC_sex}/primary_PVFC/{comparison}_primary_PVFC.csv').set_index('Accession').rename(columns={'Fold_change':f'{comparison}_Fold_change'}).filter(regex=f'{comparison}|BH')

# Scale Fold Change Data using Range Scaling
comparison_DF_min = np.min(comparison_DF[f'{comparison}_Fold_change'])
comparison_DF_max = np.max(comparison_DF[f'{comparison}_Fold_change'])
comparison_DF_mean = np.mean(comparison_DF[f'{comparison}_Fold_change'])
comparison_DF[f'{comparison}_Fold_change'] = comparison_DF[f'{comparison}_Fold_change'] - comparison_DF_mean
comparison_DF[f'{comparison}_Fold_change'] = comparison_DF[f'{comparison}_Fold_change'] / (comparison_DF_max - comparison_DF_min)

# Reverser Log2 the data for calculations
comparison_DF[f'{comparison}_Fold_change'] = 2**comparison_DF[f'{comparison}_Fold_change']

# Create a list that contains the protein ID used to match the protein names in the String output
protein_list = comparison_DF.index.to_list()
for position,protein in enumerate(protein_list):
  protein = protein.split('|')[0]
  protein_list[position] = protein

comparison_DF['Accession'] = protein_list

comparison_DF = comparison_DF.set_index('Accession')

full_comparison_DF = full_comparison_DF[[f'{comparison}_Fold_change','Protein_ID','Accession']].dropna()

STRING_ID_MAP = pd.read_csv('/content/drive/MyDrive/ApoE Analysis January-11-2023/String_ID_Map.tsv', sep='\t').rename(columns={'From':'Accession','To':'Protein_ID'})
STRING_ID_MAP = STRING_ID_MAP.sort_values(by='Accession')
STRING_ID_MAP = STRING_ID_MAP.drop_duplicates('Accession').set_index('Accession')

full_comparison_DF = pd.concat([full_comparison_DF,STRING_ID_MAP],axis=1).dropna()

# Read string DB output CSV
string_annot = pd.read_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/String All Genotype Analysis.tsv',sep='\t')
string_annot['matching proteins in your network (labels)'] = string_annot['matching proteins in your network (labels)'][:].str.split(r',')

string_annot['Ontology_coverage (%)'] = string_annot['observed gene count']/string_annot['background gene count']
string_annot = string_annot.loc[string_annot['Ontology_coverage (%)'] >= 0.25,]

ontologies_of_interest = ['GO Process','GO Function','GO Component','KEGG','Reactome','WikiPathways']
string_annot = string_annot.loc[string_annot['#category'].isin(ontologies_of_interest)].reset_index(drop=True)

string_annot[[f'{comparison}']] = None
string_annot[f'measurement_count'] = None
string_annot[f'sig_proteins_in_ont'] = None
string_annot[f'sig_proteins_count'] = None


# Create a dataframe column that contains all protein fold changes from the ontology
counter = 0
while counter < len(string_annot.index):
  protein_list = string_annot.loc[counter,'matching proteins in your network (labels)']
  protein_in_ontology = full_comparison_DF.loc[full_comparison_DF['Protein_ID'].isin(protein_list)]
  string_annot[f'{comparison}'][counter] = (protein_in_ontology[f'{comparison}_Fold_change'].values.tolist())
  string_annot.loc[counter,f'measurement_count'] = len((protein_in_ontology[f'{comparison}_Fold_change'].values.tolist()))
  counter += 1

# Create a column with a list of proteins that have significant change in concentration
sig_proteins = full_comparison_DF.loc[full_comparison_DF['-10*log10(BH)'] > np.log10(0.05)*-10, ]

counter = 0
while counter < len(string_annot.index):

  protein_list = string_annot.loc[counter,'matching proteins in your network (labels)']
  sig_protein_in_ontology = sig_proteins.loc[sig_proteins['Protein_ID'].isin(protein_list)].index.to_list()
  string_annot[f'sig_proteins_in_ont'][counter] = sig_protein_in_ontology
  string_annot.loc[counter,f'sig_proteins_count'] = len(sig_protein_in_ontology)
  counter += 1

string_annot[f'sig_proteins_ont_coverage'] = string_annot['sig_proteins_count']/string_annot['background gene count']

# Create a column for average ontology fold change and p-value calculated from all protein fold changes.
string_annot[f'{comparison}_PV_from_Ontology_FC'] = None
string_annot[f'{comparison}_Avg_Ontology_FC'] = None

# Use a one-sample t-test for fold changes in the ontology (log2(FC) != 1) and calculate FC average for the ontology
counter = 0
while counter < len(string_annot.index):
  if len(string_annot[f'{comparison}'][counter]) > 1:

    string_annot.loc[counter,f'{comparison}_Avg_Ontology_FC'] = np.mean(string_annot[f'{comparison}'][counter])
    string_annot.loc[counter,f'{comparison}_PV_from_Ontology_FC'] = st.ttest_1samp((string_annot[f'{comparison}'][counter]), 1 , nan_policy='omit', alternative='two-sided')[1]
    counter += 1
  else:
    counter += 1

# Log transform FCs and PVs
string_annot[f'{comparison}_log_PV'] = np.log10(string_annot[f'{comparison}_PV_from_Ontology_FC'].astype(float))*-10
string_annot[f'{comparison}_log_FC'] = np.log2(string_annot[f'{comparison}_Avg_Ontology_FC'].astype(float))*10

string_annot = string_annot.dropna(subset=[f'{comparison}_log_PV'])
string_annot = string_annot.sort_values(by=f'{comparison}_PV_from_Ontology_FC')
string_annot = string_annot.drop_duplicates(subset=['term ID'], keep='first')

# Calculate BH P-value correction
def performBH_correction(input_df,comparison=None):
  input_df[f'{comparison}_BH_from_Ontology_FC'] = None
  input_df = input_df.loc[input_df[f'{comparison}_log_PV'] > 0,]
  input_df = input_df.reset_index()
  #sort cleaned_df by pvalue jc mod
  input_df = input_df.sort_values(by=f'{comparison}_PV_from_Ontology_FC')
  input_df = input_df.reset_index(drop=True) #sort keeps the origional index value so you need to re-index to use it in the BH calc
  #calculate benjamini_hochberg correction as (rank/total numer of tests)*probability of false positive jc mod
  total_rows = len(input_df.index)
  for row in input_df.itertuples():
    BH_pval = ((row.Index+1)/total_rows)*0.25
    input_df.at[row.Index, f'{comparison}_BH_from_Ontology_FC'] = BH_pval
  input_df[f'{comparison}_-10*log10(BH)'] = np.log10(input_df[f'{comparison}_BH_from_Ontology_FC'].astype(float))*-10
  return(input_df)

Ontology_Comparison = performBH_correction(string_annot,comparison=f'{comparison}')

Ontology_Comparison.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/Male-Female/String_Ontology_Stats/{comparison}_SDB_Stats.csv')

Ontology_Comparison

In [ ]:
 def create_Rate_DF(comparison=comparison):

  protein_ID_guide = pd.read_csv('/content/drive/MyDrive/ApoE Turnover Data/Protein_ID_Guide/Protein_ID_Guide_File.csv').set_index('Accession')

  # Create dataframes for rates in each isoform
  a2_df = pd.DataFrame()
  a3_df = pd.DataFrame()
  a4_df = pd.DataFrame()

  # Where are the Rate files located
  where_are_the_files = f'/content/drive/MyDrive/ApoE Turnover Data/Female_Rates_Peptide_FDR_2'
  os.chdir(where_are_the_files)
  all_file_names = [i for i in glob.glob('*.{}'.format('csv'))]

  full_bkg_df = pd.DataFrame()

  # Filter each rate file and create allele specific dataframes
  for dataSet in all_file_names:
    fileName = dataSet.strip('.csv')
    TR_df = pd.read_csv(dataSet).set_index('analyte_id').filter(regex='Combined')

    TR_df = TR_df.fillna(0)
    TR_df = TR_df.loc[TR_df['Combined uniques'].astype(int) > 1,]
    TR_df = TR_df.loc[TR_df['Combined rate'] != 'Insufficient Timepoints',]
    TR_df = TR_df.loc[TR_df['Combined rate'] != 'value could not be determined',]
    TR_df = TR_df.loc[TR_df['Combined rate'].astype(float) > 0,]
    TR_df = TR_df.loc[TR_df['Combined R2'].astype(float) > 0.6,]
    TR_df = TR_df.filter(regex='rate')

    allele = re.findall('A\d|M\d', fileName)[0]
    TR_df = TR_df.rename(columns={'Combined rate' : f'{allele}'})
    if allele == 'A2':
      a2_df = pd.concat([a2_df,TR_df])
    elif allele == 'A3':
      a3_df = pd.concat([a3_df,TR_df])
    elif allele == 'A4':
      a4_df = pd.concat([a4_df,TR_df])


  # Calculate the average rate for each protein from the different rate files
  a2_df['A2'] = a2_df['A2'].astype(float)
  a2_df = a2_df.groupby(by='analyte_id',as_index=True).mean()
  a2_std = np.std(np.log2(a2_df['A2']))
  a2_df['A2'] = np.log2(a2_df['A2'])

  a3_df['A3'] = a3_df['A3'].astype(float)
  a3_df = a3_df.groupby(by='analyte_id',as_index=True).mean()
  a3_std = np.std(np.log2(a3_df['A3']))
  a3_df['A3'] = np.log2(a3_df['A3'])

  a4_df['A4'] = a4_df['A4'].astype(float)
  a4_df = a4_df.groupby(by='analyte_id',as_index=True).mean()
  a4_std = np.std(np.log2(a4_df['A4']))
  a4_df['A4'] = np.log2(a4_df['A4'])

  # Create a dataframe with all the rates and
  all_rates = pd.concat([a2_df,a3_df,a4_df,protein_ID_guide],axis=1)

  # Create comparison specific dataframes
  E2vE3_rates_df = pd.concat([a2_df,a3_df,protein_ID_guide],axis=1).dropna()
  E4vE3_rates_df = pd.concat([a3_df,a4_df,protein_ID_guide],axis=1).dropna()
  E4vE2_rates_df = pd.concat([a2_df,a4_df,protein_ID_guide],axis=1).dropna()

  # Calculate fold change
  E2vE3_rates_df['E2vE3_rate_FC'] = ((E2vE3_rates_df['A2'] - E2vE3_rates_df['A3']) )
  E4vE3_rates_df['E4vE3_rate_FC'] = ((E4vE3_rates_df['A4'] - E4vE3_rates_df['A3']) )
  E4vE2_rates_df['E4vE2_rate_FC'] = ((E4vE2_rates_df['A4'] - E4vE2_rates_df['A2']) )

  # Concatenate the the comparison dataframes
  Rate_FC_DF = pd.concat([E2vE3_rates_df['E2vE3_rate_FC'],E4vE3_rates_df['E4vE3_rate_FC'],E4vE2_rates_df['E4vE2_rate_FC'],protein_ID_guide],axis=1)

  # norm = Rate_FC_DF.plot.density(figsize=(10, 10),)
  # plt.title('Rate Fold Change Density Plot')
  # plt.xlim(-6,6)

  Rate_FC_DF.to_csv('/content/drive/MyDrive/ApoE Turnover Data/Rate_FC_DF')

  # full_Rate_FC_df = Rate_FC_DF.filter(regex=f'{comparison}|ID')
  full_Rate_FC_df = Rate_FC_DF

  print('This function outputs a rate FC dataframe; the FC was calculated with the log2 rate values, FC = [log2(B) - log(A)]')
  # display(full_Rate_FC_df)

  return(full_Rate_FC_df)

full_Rate_FC_df = create_Rate_DF(comparison=comparison)

Rate_FC_df = full_Rate_FC_df.copy()
Rate_FC_df = Rate_FC_df.filter(regex=f'{comparison}|ID').dropna()

def auto_scaling(df):
  Rate_FC_DF_mean = np.mean(df[f'{comparison}_rate_FC'])
  Rate_FC_DF_std = np.std(df[f'{comparison}_rate_FC'])
  df[f'{comparison}_rate_FC'] = df[f'{comparison}_rate_FC'] - Rate_FC_DF_mean
  df[f'{comparison}_rate_FC'] = df[f'{comparison}_rate_FC'] / np.sqrt(Rate_FC_DF_std)
  return(df)

scaled_RDF = auto_scaling(Rate_FC_df).filter(regex='rate')

STRING_ID_MAP = pd.read_csv('/content/drive/MyDrive/ApoE Analysis January-11-2023/String_ID_Map.tsv', sep='\t').rename(columns={'From':'Accession','To':'Protein_ID'})
STRING_ID_MAP = STRING_ID_MAP.sort_values(by='Accession')
STRING_ID_MAP = STRING_ID_MAP.drop_duplicates('Accession').set_index('Accession')

scaled_RDF =scaled_RDF.copy()

scaled_RDF = pd.concat([scaled_RDF,STRING_ID_MAP],axis=1).dropna()

full_Rate_FC_df = scaled_RDF.copy()

# Format the string file to create readable proteins lists for each ontology
string_annot = pd.read_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/String All Genotype Analysis.tsv',sep='\t')
string_annot['matching proteins in your network (labels)'] = string_annot['matching proteins in your network (labels)'][:].str.split(r',')

string_annot['Ontology_coverage (%)'] = string_annot['observed gene count']/string_annot['background gene count']
string_annot = string_annot.loc[string_annot['Ontology_coverage (%)'] >= 0.25,]

ontologies_of_interest = ['GO Process','GO Function','GO Component','KEGG','Reactome','WikiPathways']
string_annot = string_annot.loc[string_annot['#category'].isin(ontologies_of_interest)].reset_index(drop=True)
string_annot[[f'{comparison}_Rate']] = None

# Create a dataframe column that contains all protein fold changes from the ontology
counter = 0
while counter < len(string_annot.index):
  protein_list = string_annot.loc[counter,'matching proteins in your network (labels)']
  protein_in_ontology = full_Rate_FC_df.loc[full_Rate_FC_df['Protein_ID'].isin(protein_list)]
  string_annot[f'{comparison}_Rate'][counter] = (protein_in_ontology[f'{comparison}_rate_FC'].dropna()).values.tolist()
  counter += 1

# Create a column for average ontology fold change and p-value calculated from all protein fold changes.
counter = 0
string_annot[f'{comparison}_PV_from_Ontology_Rate_FC'] = None
string_annot[f'{comparison}_PV_from_Ontology_Rate_FC'] = None

string_annot[f'{comparison}_Avg_Ontology_Rate_FC'] = None
string_annot[f'{comparison}_Avg_Ontology_Rate_FC'] = None

# Use a one-sample t-test for fold changes in the ontology (FC != 1) and calculate FC average for the ontology
counter = 0
while counter < len(string_annot.index):
  if len(string_annot[f'{comparison}_Rate'][counter]) > 2:
    string_annot.loc[counter,f'{comparison}_PV_from_Ontology_Rate_FC'] = st.ttest_1samp((string_annot[f'{comparison}_Rate'][counter]), 0 , nan_policy='omit', alternative='two-sided')[1]
    string_annot.loc[counter,f'{comparison}_Avg_Ontology_Rate_FC'] = np.mean(string_annot[f'{comparison}_Rate'][counter])
    counter += 1
  else:
    counter += 1

rate_string_analysis = string_annot.copy()

rate_string_analysis[f'{comparison}_Rate_log_PV'] = np.log10(rate_string_analysis[f'{comparison}_PV_from_Ontology_Rate_FC'].astype(float))*-10
rate_string_analysis[f'{comparison}_Rate_log_FC'] = (rate_string_analysis[f'{comparison}_Avg_Ontology_Rate_FC'].astype(float))

rate_string_analysis = rate_string_analysis.dropna(subset=[f'{comparison}_Rate_log_PV'])
rate_string_analysis = rate_string_analysis.sort_values(by=f'{comparison}_PV_from_Ontology_Rate_FC')

# Calculate BH P-value correction
def performBH_correction(input_df,comparison=None):
  input_df[f'{comparison}_BH_from_Ontology_Rate_FC'] = None
  input_df = input_df.loc[input_df[f'{comparison}_Rate_log_PV'] > 0,]
  input_df = input_df.reset_index()
  input_df = input_df.sort_values(by=f'{comparison}_PV_from_Ontology_Rate_FC')
  input_df = input_df.reset_index(drop=True) #sort keeps the origional index value so you need to re-index to use it in the BH calc
  #calculate benjamini_hochberg correction as (rank/total numer of tests)*probability of false positive jc mod
  total_rows = len(input_df.index)
  for row in input_df.itertuples():
    BH_pval = ((row.Index+1)/total_rows)*0.25
    input_df.at[row.Index, f'{comparison}_BH_from_Ontology_Rate_FC'] = BH_pval
  input_df[f'{comparison}_Rate_-10*log10(BH)'] = np.log10(input_df[f'{comparison}_BH_from_Ontology_Rate_FC'].astype(float))*-10
  return(input_df)

rate_string_analysis = performBH_correction(rate_string_analysis,comparison=f'{comparison}')

rate_string_analysis.to_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/Male-Female/String_Ontology_Stats/{comparison}_SDB_Rate_Stats.csv')

In [ ]:
List_Dictionary_E2vE3 = {'Mitochondrial Components':['GO:0031966','GO:0005743','GO:1990542','GO:0098798','GO:0030150','GO:0005759'],
                         'OXPHOS':['MMU-1428517','GO:0022900','GO:0033177','MMU-204174','GO:0005967','mmu00020'],
                         'Proteasome':['GO:0005839'],
                         'Endolysosomal Pathway':['GO:0005765','GO:0005764','GO:0007041','GO:0007040']
                         }


List_Dictionary_E4vE3 = {'Mitochondrial Components':['GO:0031966','GO:0005743','GO:1990542','GO:0098573','GO:0098798','GO:0031305','GO:0005742','GO:0098800'],
                         'OXPHOS':['GO:0022904','MMU-1428517','GO:0009060','mmu00051'],
                         'Proteasome':['GO:0030544','GO:0008541','GO:0005838','GO:0022624','GO:0000502'],
                         'Endolysosomal Pathway':['GO:0005765','GO:0005764','MMU-8856828','GO:0005770','GO:0031902']
                         }

In [ ]:
# This code concatenates the rate string analysis and the concentration string analysis

def create_RNC_DF(comparison=None):

  # Read the concentration SBD analysis
  SDB_conc_Stats = pd.read_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/Male-Female/String_Ontology_Stats/{comparison}_SDB_Stats.csv')
  SDB_conc_Stats['Ontology_coverage (%)'] = SDB_conc_Stats['observed gene count']/SDB_conc_Stats['background gene count']
  SDB_conc_Stats = SDB_conc_Stats.set_index('term ID').filter(regex='E\dvE\d_log_FC|term.description|PV|labels|observed|BH|coverage|sig').rename(columns={f'{comparison}' : f'{comparison}_FCs',f'{comparison}_log_FC' : f'{comparison}_Conc_log_FC'})

  # Read the Rate SBD analysis
  SDB_rate_Stats = pd.read_csv(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/Male-Female/String_Ontology_Stats/{comparison}_SDB_Rate_Stats.csv')
  SDB_rate_Stats = SDB_rate_Stats.set_index('term ID').filter(regex=f'{comparison}_Rate_log_FC|category')
  SDB_rate_Stats[f'{comparison}_Rate_log_FC'] = SDB_rate_Stats[f'{comparison}_Rate_log_FC']

  # Combined the Concentration and Rate SDB analysis
  RnC_Df = pd.concat([SDB_conc_Stats,SDB_rate_Stats],axis=1).dropna()
  RnC_Df = RnC_Df.rename(columns={'observed gene count':'Ontology Protein Count'})
  RnC_Df["term description (Count)"] = (RnC_Df['term description']).astype(str) + " " +"(" + (RnC_Df['Ontology Protein Count'].astype(int)).astype(str) + ")"
  RnC_Df = RnC_Df.filter(regex='E\dvE\d_log_FC|term.description.|labels|observed|10.BH|coverage|log_FC|count|category|sig')

  RnC_Df = RnC_Df.rename(columns={f'sig_proteins_in_ont':f'{comparison}_sig_proteins_in_ont'})
  RnC_Df = RnC_Df.rename(columns={f'sig_proteins_count':f'{comparison}_sig_proteins_count'})

  RnC_Df = RnC_Df[['#category',
                  'term description (Count)',
                  f'{comparison}_Conc_log_FC',
                  f'{comparison}_Rate_log_FC',
                  f'{comparison}_-10*log10(BH)',
                  'Ontology_coverage (%)',
                  'matching proteins in your network (labels)',
                  f'{comparison}_sig_proteins_in_ont',
                  f'{comparison}_sig_proteins_count']]


  return(RnC_Df)

RnC_Df_for_lists_E2vE3 = create_RNC_DF(comparison='E2vE3')
RnC_Df_for_lists_E4vE3 = create_RNC_DF(comparison='E4vE3')
RnC_Df_for_lists_E4vE2 = create_RNC_DF(comparison='E4vE2')

In [ ]:
#@title
# colors = ['#646464','#306998','#FFD43B']
colors = ['#00375F','#730707']
custom_pal = sns.color_palette(colors)

def plot_Quadrants(input_df,
           plot_title=None,
           Conc_column_name=None,
           Rate_column_name=None,
           fractions=None,
           comparison=None,
           style=None,
           list_of_interest=None,
           Ontology_name=None):

  df = input_df.copy().filter(regex='log.FC|term|labels|observed|BH')

  plt.figure(figsize=(3,3))

  custom_params = {'axes.linewidth': 1}

  sns.set(style='ticks', context='paper', color_codes=True,rc=custom_params)
  # sns.set(style='ticks', context='paper', font_scale=1, color_codes=True,rc=custom_params)

  plt.axvline(x=0, ymin=0, ymax=1, linestyle=':',color='gray',linewidth=2)
  plt.axhline(y=0, xmin=0, xmax=1, linestyle=':',color='gray',linewidth=2)
  plt.xlabel("ΔConcentration (arb. unit)", fontweight ='bold', size=10)
  plt.ylabel("ΔRate (arb. unit)", fontweight ='bold',size=10)
  plt.title(f'{comparison} Ontology ΔConcentration vs ΔRate',fontweight ='bold',size=10, pad=5, wrap=False)
  plt.tick_params(axis='both', which='major', labelsize=10,length = 2,color = 'black',width =1)

  df["significance"] = "Insignificant"
  df.loc[(df[f'{comparison}_-10*log10(BH)'] > np.log10(0.025)*-10), 'significance'] = "Significant"
  df.sort_values(by=f'{comparison}_-10*log10(BH)',inplace=True)

  sig_len = len(df.loc[(df["significance"] == 'Significant'),])
  insig_len = len(df.loc[(df["significance"] == 'Insignificant'),])

  df["significance"] = f"Insignificant ({insig_len})"
  df.loc[(df[f'{comparison}_-10*log10(BH)'] > np.log10(0.025)*-10), 'significance'] = f"Significant ({sig_len})"
  df.sort_values(by=f'{comparison}_-10*log10(BH)',inplace=True)


  VP = sns.scatterplot(x=f'{comparison}_Conc_log_FC',
                      y=f'{comparison}_Rate_log_FC',
                      data=df,
                      legend=True,
                      palette=custom_pal,
                      # c='#00375F',
                      hue='significance',
                      s = 50,
                      alpha=0.7,
                      edgecolor="black",
                      linewidth=0.4,
                      hue_order = [f"Insignificant ({insig_len})",f"Significant ({sig_len})"],
                      style=style)

  plt.legend(loc=8, prop={'size':8},title_fontsize=10,markerscale=.75,borderpad=0.1,borderaxespad=-5, ncols=2)

  # ax.legend(prop={'size':15},title_fontsize=20,markerscale=3,borderpad=0.5,borderaxespad=-5, ncols=2)

  # ax.legend(prop={'size':15},title_fontsize=20,markerscale=3,borderpad=0.5,loc=8,borderaxespad=-5, ncols=2)

  sns.despine(top=True, right=True, left=False, bottom=False, offset=None, trim=False)

  plt.savefig(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{FC_sex}/String_Ontology_Stats/{comparison}_Proteostasis_Plot.svg',format="svg",transparent=True, dpi=1200,bbox_inches='tight')

  plt.show()

  return(VP)


plot_Quadrants(RnC_Df_for_lists_E2vE3,
              plot_title='Rate vs Concentration',
              Conc_column_name=f'E2vE3_log_FC',
              Rate_column_name=f'E2vE3_Rate_log_FC',
              fractions=f'E2vE3',
              comparison=f'E2vE3',
              style=None,
              list_of_interest=None,
              Ontology_name=None)

plot_Quadrants(RnC_Df_for_lists_E4vE3,
              plot_title='Rate vs Concentration',
              Conc_column_name=f'E4vE3_log_FC',
              Rate_column_name=f'E4vE3_Rate_log_FC',
              fractions=f'E4vE3',
              comparison=f'E4vE3',
              style=None,
              list_of_interest=None,
              Ontology_name=None)

In [ ]:
# This creates dictionary with Ontology category as the keys and the proteins associated with those ontology categories

def Ontology_protein_dictionary(SDB_conc_Stats=None,comparison=None):
  if comparison == 'E2vE3':
    List_Dictionary = List_Dictionary_E2vE3
  elif comparison == 'E4vE3':
    List_Dictionary = List_Dictionary_E4vE3

  SDB_conc_Stats = SDB_conc_Stats.reset_index()

  All_Ontology_Genes_list = []

  Ontology_protein_dictionary = {}

  for ontology_category,ontologies in List_Dictionary.items():
    # print(ontology_category)

    Ontology_category_protein_list = []

    for ontology in ontologies:
      print(ontology)
      ontology_proteins = SDB_conc_Stats.loc[SDB_conc_Stats['term ID'] == ontology,'matching proteins in your network (labels)'].values.tolist()[0]
      # print(ontology_proteins)
      ontology_proteins = ontology_proteins[:].strip(r'[|]')
      ontology_proteins = ontology_proteins[:].replace(" ", "")
      ontology_proteins = ontology_proteins[:].replace("'", "")
      ontology_proteins = ontology_proteins[:].split(r',')
      Ontology_category_protein_list = Ontology_category_protein_list + ontology_proteins

    Proteins_in_ontology_category = list(dict.fromkeys(Ontology_category_protein_list))
    Proteins_in_ontology_category = sorted(Proteins_in_ontology_category)
    ontology_category_protein_count = len(Proteins_in_ontology_category)

    All_Ontology_Genes_list = Proteins_in_ontology_category + All_Ontology_Genes_list

    Ontology_protein_dictionary[ontology_category] = Proteins_in_ontology_category

  all_Proteins_in_ontology_category = list(dict.fromkeys(All_Ontology_Genes_list))
  all_Proteins_in_ontology_category = sorted(all_Proteins_in_ontology_category)
  all_ontology_category_protein_count = len(all_Proteins_in_ontology_category)

  return(Ontology_protein_dictionary)

Ontology_protein_dictionary_E2vE3 = Ontology_protein_dictionary(SDB_conc_Stats=RnC_Df_for_lists_E2vE3,comparison='E2vE3')
Ontology_protein_dictionary_E4vE3 = Ontology_protein_dictionary(SDB_conc_Stats=RnC_Df_for_lists_E4vE3,comparison='E4vE3')

In [ ]:
from textwrap import wrap

def bar_plot_for_ontologies(input_df,list_of_interest,Ontology_name=None,comparison=None):

  df = input_df.copy()
  df = df.reset_index()
  df = df.set_index(df['term ID'])

  ontologies_of_interest = df.loc[df['term ID'].isin(list(list_of_interest)),]
  ontologies_of_interest = ontologies_of_interest.drop_duplicates(subset='term ID')
  ontologies_of_interest = ontologies_of_interest.set_index('term description (Count)')
  ontologies_of_interest.sort_values(by=f'{comparison}_Conc_log_FC', inplace=True, ascending=True)

  first_bar = ontologies_of_interest[f'{comparison}_Conc_log_FC']
  first_bar_label = 'ΔConcentration'
  first_bar_color = '#EA7600'
  second_bar = ontologies_of_interest[f'{comparison}_Rate_log_FC']
  second_bar_label = 'ΔRate'
  second_bar_color = '#236192'
  labels = ontologies_of_interest.index
  labels = [ '\n'.join(wrap(l, 30)) for l in labels ]
  width = 0.5  # the width of the bars
  plot_title = f'{Ontology_name} Ontologies \n ΔConcentration and ΔRate\n{comparison}'
  title_size = 10

  # plt.suptitle(f'{Ontology_name}', y=1.05, fontsize=20)

  # Sort values for plotting

  # Plot figure
  # fig, ax = plt.subplots(figsize=(10,15),linewidth=10)
  fig, ax = plt.subplots(linewidth=2)

  ax.tick_params(width=2)
  plt_size = len(labels)*0.5
  fig.set_figheight(plt_size+1)
  # fig.set_figwidth(plt_size+1)
  plt.tight_layout()

  # Plot double bars
  y = np.arange(len(labels))  # the label locations
  ax.barh(y + width/2, first_bar, width, label=first_bar_label, color=first_bar_color)
  ax.barh(y - width/2, second_bar, width, label=second_bar_label, color=second_bar_color)

  # Format ticks
  ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.2f}'))

  # Create labels
  rects = ax.patches
  for rect in rects:
      # Get X and Y placement of label from rect.
      x_value = rect.get_width()
      y_value = rect.get_y() + rect.get_height() / 2
      space = 1
      ha = 'left'
      if x_value < 0:
          space *= 1
          ha = 'right'
      label = '{:,.2f}'.format(x_value)
      plt.annotate(
          label,
          (x_value, y_value),
          xytext=(space, 1),
          textcoords='offset points',
          va='center',
          ha=ha,fontsize=10)

  # plt.xticks(rotation=45)
  # Set y-labels and legend

  ax.set_yticklabels(labels)
  ax.legend(prop={'size':10},title_fontsize=10,markerscale=3,borderpad=0.5,loc=8,borderaxespad=-5, ncols=2)

  # To show each y-label, not just even ones
  plt.yticks(np.arange(min(y), max(y)+1, 1.0))

  # Adjust subplots
  plt.margins(x=0.1)
  plt.subplots_adjust(left=0.35, top=0.9)

  plt.xlabel("Change (arb. units)", fontweight ='bold', size=10)
  plt.ylabel("Protein Ontology\n(Protein Count)", fontweight ='bold',size=10, wrap=False)
  plt.tick_params(axis='both', which='major', labelsize=8,length = 2,color = 'black',width =2)
  plt.tick_params(axis='y', which='major', labelsize=8,length = 1,color = 'black',width =10)

  # Set title
  title = plt.title(plot_title,fontweight ='bold',size=10, pad=20, wrap=False)
  title.set_position([.35, 1])

  # Set subtitle
  tform = ax.get_xaxis_transform()

  ax.spines['right'].set_visible(False)
  ax.spines['top'].set_visible(False)
  ax.spines['left'].set_visible(True)
  ax.spines['bottom'].set_visible(True)

  plt.savefig(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{FC_sex}/String_Ontology_Stats/{ontology_category}_{comparison}_RnC_Plot.svg',format="svg",transparent=True,bbox_inches='tight')


for ontology_category,ontologies in List_Dictionary_E2vE3.items():
  print(ontology_category)
  bar_plot_for_ontologies(input_df=RnC_Df_for_lists_E2vE3,
                          list_of_interest=ontologies,
                          Ontology_name=ontology_category,comparison='E2vE3')

for ontology_category,ontologies in List_Dictionary_E4vE3.items():
  print(ontology_category,ontologies)
  bar_plot_for_ontologies(input_df=RnC_Df_for_lists_E4vE3,
                          list_of_interest=ontologies,
                          Ontology_name=ontology_category,comparison='E4vE3')

In [ ]:
ont_Dictionary = {'Mitochondrial Components':['GO:0031966','GO:0005743','GO:1990542','GO:0098798','GO:0030150','GO:0005759','GO:0031966','GO:0098573','GO:0031305','GO:0005742','GO:0098800'],
                         'OXPHOS':['MMU-1428517','GO:0022900','GO:0033177','MMU-204174','GO:0005967','mmu00020','GO:0022904','MMU-1428517','GO:0009060','mmu00051'],
                         'ProteasomeLysosome':['GO:0005839','GO:0030544','GO:0008541','GO:0005838','GO:0022624','GO:0000502'],
                          'Endolysosomal Pathway':['GO:0005765','GO:0005764','MMU-8856828','GO:0005770','GO:0031902','GO:0007041','GO:0007040']
                         }

def Ontology_protein_dictionary(SDB_conc_Stats=None,comparison=None):
  if comparison == 'E2vE3':
    List_Dictionary = ont_Dictionary
  elif comparison == 'E4vE3':
    List_Dictionary = ont_Dictionary

  SDB_conc_Stats = SDB_conc_Stats.reset_index()
  Ontology_protein_dictionary = {}

  for ontology_category,ontologies in List_Dictionary.items():

    shared_counts_df = pd.DataFrame(None)
    Ontology_protein_dictionary = {}

    for ontology in ontologies:
      ontology_name = SDB_conc_Stats.loc[SDB_conc_Stats['term ID'] == ontology,'term description (Count)'].values.tolist()[0]
      ontology_proteins = SDB_conc_Stats.loc[SDB_conc_Stats['term ID'] == ontology,f'matching proteins in your network (labels)'].values.tolist()[0]
      ontology_proteins = ontology_proteins[:].strip(r'[|]')
      ontology_proteins = ontology_proteins[:].replace(" ", "")
      ontology_proteins = ontology_proteins[:].replace("'", "")
      ontology_proteins = ontology_proteins[:].split(r',')
      Ontology_protein_dictionary[f'{ontology_name}'] = ontology_proteins

    protein_lists = list(Ontology_protein_dictionary.values())
    ontology_list = list(Ontology_protein_dictionary.keys())

    def calculate_shared_percentage(list_dict):
        list_names = list(list_dict.keys())
        shared_percentages = {}
        for i in range(len(list_names)):

            for j in range(i + 1, len(list_names)):
                list1_set = set(list_dict[list_names[i]])
                list2_set = set(list_dict[list_names[j]])
                shared_items = len(list1_set & list2_set)
                total_items = len(list1_set | list2_set)
                shared_percentages[(i, j)] = (shared_items / total_items) * 100
            shared_percentages[(i, i)] = 100

        return shared_percentages

    def create_heatmap(shared_percentages, list_names):
        matrix = [[0 for _ in range(len(list_names))] for _ in range(len(list_names))]
        for (i, j), percentage in shared_percentages.items():
            matrix[i][j] = percentage
            matrix[j][i] = percentage
        return matrix


    shared_percentages = calculate_shared_percentage(Ontology_protein_dictionary)
    list_names = list(Ontology_protein_dictionary.keys())

    # Create the heatmap data
    heatmap_data = create_heatmap(shared_percentages, list_names)
    heatmap_df = pd.DataFrame(heatmap_data, index=list_names, columns=list_names)

    labels = heatmap_df.index
    labels = [ '\n'.join(wrap(l, 35)) for l in labels ]

    sns.heatmap(heatmap_df, annot=True, cmap='crest', fmt='.0f', linewidths = 0.30, robust=True,label=labels,xticklabels=labels,yticklabels=labels)

    # plt.xlabel('Ontology', fontweight ='bold', size=20)
    # plt.ylabel('Ontology', fontweight ='bold', size=20)
    plt.title('Ontology Shared Proteins (%)',fontweight ='bold',size=10, pad=15, wrap=False)
    plt.tick_params(axis='both', which='major', labelsize=10,length = 5,color = 'black',width =3)
    plt.savefig(f'/content/drive/MyDrive/ApoE Analysis January-11-2023/{FC_sex}/String_Ontology_Stats/{ontology_category}_HEATMAP_Plot.svg',format="svg",transparent=True,bbox_inches='tight')
    plt.show()

Ontology_protein_dictionary_E2vE3 = Ontology_protein_dictionary(SDB_conc_Stats=RnC_Df_for_lists_E2vE3,comparison='E2vE3')